In [14]:
import os
import sys
import numpy as np
import torch as th
import matplotlib.pyplot as plt
import motornet as mn


## Arm210 effector

In [ ]:
def build_arm210(verbose=False):
    
    arm210 = mn.effector.Effector(
    skeleton=mn.skeleton.TwoDofArm(),
    muscle=mn.muscle.RigidTendonHillMuscleThelen(),
    integration_method='rk4'
    )

    arm210.add_muscle(
    path_fixation_body=[0., 1.],
    path_coordinates=[[-.15, .03], [.094, 0.017]],
    name='pectoralis',
    max_isometric_force=838,
    tendon_length=.039,
    optimal_muscle_length=.134,
    normalized_slack_muscle_length=1.48)

    arm210.add_muscle(
    path_fixation_body=[0., 1.],
    path_coordinates=[[-.034, .022], [.144, 0.01]],
    name='clavicular deltoid',
    max_isometric_force=680,
    tendon_length=.039,
    optimal_muscle_length=.104,
    normalized_slack_muscle_length=1.4)

    arm210.add_muscle(
    path_fixation_body=[0., 0., 1.],
    path_coordinates=[[.14, 0.], [.05, -.00], [0.153, 0.]],
    name='deltoid',
    max_isometric_force=1207,
    tendon_length=.066,
    optimal_muscle_length=.140,
    normalized_slack_muscle_length=1.52)

    arm210.add_muscle(
    path_fixation_body=[0., 0., 1.],
    path_coordinates=[[.1, 0.], [.05, -.03], [0.062, 0.004]],
    name='teres major',
    max_isometric_force=1207,
    tendon_length=.066,
    optimal_muscle_length=.068,
    normalized_slack_muscle_length=1.65)

    arm210.add_muscle(
    path_fixation_body=[1., 2.],
    path_coordinates=[[0.23, 0.001], [0.231, 0.01]],
    name='brachioradialis',
    max_isometric_force=1422,
    tendon_length=.172,
    optimal_muscle_length=.092,
    normalized_slack_muscle_length=1.43)

    arm210.add_muscle(
    path_fixation_body=[1., 1., 2.],
    path_coordinates=[[0.03, 0.], [0.138, -0.019], [-0.04, -0.017]],
    name='tricepslat',
    max_isometric_force=1549,
    tendon_length=.187,
    optimal_muscle_length=.093,
    normalized_slack_muscle_length=1.45)

    arm210.add_muscle(
    path_fixation_body=[0., 2.],
    path_coordinates=[[-0.052, 0.033], [0.044, 0.001]],
    name='biceps',
    max_isometric_force=414,
    tendon_length=.204,
    optimal_muscle_length=.137,
    normalized_slack_muscle_length=1.5)

    arm210.add_muscle(
    path_fixation_body=[0., 2.],
    path_coordinates=[[0.02, -0.028], [-0.04, -0.017]],
    name='tricepslong',
    max_isometric_force=603,
    tendon_length=0.217,
    optimal_muscle_length=0.127,
    normalized_slack_muscle_length=1.4)

    arm210.add_muscle(
    path_fixation_body=[1., 2.],
    path_coordinates=[[0.306, -0.011], [0.003, -0.025]],
    name='anconeus',
    max_isometric_force=300,
    tendon_length=0.01,
    optimal_muscle_length=0.015,
    normalized_slack_muscle_length=1.72)

    arm210.add_muscle(
    path_fixation_body=[1., 2.],
    path_coordinates=[[0.277, 0.], [0.075, 0.02]],
    name='prot',
    max_isometric_force=700,
    tendon_length=0.02,
    optimal_muscle_length=0.058,
    normalized_slack_muscle_length=1.48)

    if verbose:
      arm210.print_muscle_wrappings()

    return arm210

arm210 = build_arm210()

MUSCLE NAME: pectoralis
-----------------------
n_fixation_points:  2
fixation body:  [0, 1]
coordinates:  [[-0.15000000596046448, 0.029999999329447746], [0.09399999678134918, 0.017000000923871994]]
max_isometric_force:  838
tendon_length:  0.039
optimal_muscle_length:  0.134
normalized_slack_muscle_length:  1.48


MUSCLE NAME: clavicular deltoid
-------------------------------
n_fixation_points:  2
fixation body:  [0, 1]
coordinates:  [[-0.03400000184774399, 0.02199999988079071], [0.14399999380111694, 0.009999999776482582]]
max_isometric_force:  680
tendon_length:  0.039
optimal_muscle_length:  0.104
normalized_slack_muscle_length:  1.4


MUSCLE NAME: deltoid
--------------------
n_fixation_points:  3
fixation body:  [0, 0, 1]
coordinates:  [[0.14000000059604645, 0.0], [0.05000000074505806, -0.0], [0.15299999713897705, 0.0]]
max_isometric_force:  1207
tendon_length:  0.066
optimal_muscle_length:  0.14
normalized_slack_muscle_length:  1.52


MUSCLE NAME: teres major
-------------------

## State visualization methods

In [16]:
def print_effector_state(effector, batch_size):

    effector.reset(options={"batch_size": batch_size})

    for key, state in effector.states.items():
        print(key + " shape: " + " " * (10-len(key)), state.shape)
    
    print()

def print_muscle_state(effector):

    features = effector.muscle.state_name
    for n, feature in enumerate(features):
        print("feature " + str(n) + ": ", feature)

print_effector_state(arm210, 7)
print_muscle_state(arm210)

joint shape:       torch.Size([7, 4])
cartesian shape:   torch.Size([7, 4])
muscle shape:      torch.Size([7, 7, 10])
geometry shape:    torch.Size([7, 4, 10])
fingertip shape:   torch.Size([7, 2])

feature 0:  activation
feature 1:  muscle length
feature 2:  muscle velocity
feature 3:  force-length PE
feature 4:  force-length CE
feature 5:  force-velocity CE
feature 6:  force


## environment

In [17]:
env = mn.environment.RandomTargetReach(effector=arm210, max_ep_duration=1.)

## Networks

In [ ]:
class Policy(th.nn.Module):

    def __init__(self, input_dim: int, hidden_dim: int, output_dim: int, device):
        super().__init__()
        self.device = device
        self.hidden_dim = hidden_dim
        self.n_layers = 1
        
        self.gru = th.nn.GRU(input_dim, hidden_dim, 1, batch_first=True)
        self.fc = th.nn.Linear(hidden_dim, output_dim)
        self.sigmoid = th.nn.Sigmoid()

        # Apply custom initialization
        for name, param in self.named_parameters():
            if name == "gru.weight_ih_l0":
                th.nn.init.orthogonal_(param)  # Changed to orthogonal
            elif name == "gru.weight_hh_l0":
                th.nn.init.orthogonal_(param)  # Already orthogonal
            elif name == "gru.bias_ih_l0":
                th.nn.init.zeros_(param)
            elif name == "gru.bias_hh_l0":
                th.nn.init.zeros_(param)
            elif name == "fc.weight":
                th.nn.init.orthogonal_(param)  # Changed to orthogonal
            elif name == "fc.bias":
                th.nn.init.constant_(param, -10.)  # Changed to -10
            else:
                raise ValueError(f"Unexpected parameter: {name}")
        
        self.to(device)

    def forward(self, x, h0):
        y, h = self.gru(x[:, None, :], h0)
        u = self.sigmoid(self.fc(y)).squeeze(dim=1)
        return u, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(self.device)
        return hidden
    
class CriticNetwork(th.nn.Module):
    def __init__(self, input_size, device):
        super().__init__()
        self.fc1 = th.nn.Linear(input_size, 100)
        self.fc2 = th.nn.Tanh(100, 64)
        self.fc3 = th.nn.Tanh(64, 64)
        self.fc4 = th.nn.Linear(64, 1)
        self.device = device

        self.to(device)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return self.fc4(x)

## Training Networks

In [22]:
policy = Policy(env.observation_space.shape[0], 32, env.n_muscles, device='cpu')
optimizer = th.optim.Adam(policy.parameters(), lr=10**-3)

In [23]:
batch_size = 32
n_batch = 6000
losses = []
interval = 250

def l1(x, y):
  """L1 loss"""
  return th.mean(th.sum(th.abs(x - y), dim=-1))

def l2(x, y):
  """L2 loss"""
  return th.mean(th.sum(x**2 - y**2), dim=-1)

for batch in range(n_batch):
  # initialize batch
  h = policy.init_hidden(batch_size=batch_size)
  obs, info = env.reset(options={"batch_size": batch_size})
  terminated = False

  # initial positions and targets
  xy = [info["states"]["fingertip"][:, None, :]]
  tg = [info["goal"][:, None, :]]

  # simulate whole episode
  while not terminated:  # will run until `max_ep_duration` is reached
    action, h = policy(obs, h)
    obs, reward, terminated, truncated, info = env.step(action=action)

    xy.append(info["states"]["fingertip"][:, None, :])  # trajectories
    tg.append(info["goal"][:, None, :])  # targets

  # concatenate into a (batch_size, n_timesteps, xy) tensor
  xy = th.cat(xy, axis=1)
  tg = th.cat(tg, axis=1)
  loss = l1(xy, tg)  # L1 loss on position
  
  # backward pass & update weights
  optimizer.zero_grad() 
  loss.backward()
  th.nn.utils.clip_grad_norm_(policy.parameters(), max_norm=1.)  # important!
  optimizer.step()
  losses.append(loss.item())

  if (batch % interval == 0) and (batch != 0):
    print("Batch {}/{} Done, mean policy loss: {}".format(batch, n_batch, sum(losses[-interval:])/interval))

Batch 250/6000 Done, mean policy loss: 0.3383630543351173


KeyboardInterrupt: 

In [ ]:
def plot_training_log(log):
  fig, axs = plt.subplots(1, 1)
  fig.set_tight_layout(True)
  fig.set_size_inches((8, 3))

  axs.semilogy(log)

  axs.set_ylabel("Loss")
  axs.set_xlabel("Batch #")
  plt.show()

plot_training_log(losses)

## Evaluating performance

In [ ]:
plotor = mn.plotor.plot_pos_over_time

def plot_simulations(xy, target_xy):
  target_x = target_xy[:, -1, 0]
  target_y = target_xy[:, -1, 1]

  plt.figure(figsize=(10,3))

  plt.subplot(1,2,1)
  plt.ylim([-1.1, 1.1])
  plt.xlim([-1.1, 1.1])
  plotor(axis=plt.gca(), cart_results=xy)
  plt.scatter(target_x, target_y)

  plt.subplot(1,2,2)
  plt.ylim([-2, 2])
  plt.xlim([-2, 2])
  plotor(axis=plt.gca(), cart_results=xy - target_xy)
  plt.axhline(0, c="grey")
  plt.axvline(0, c="grey")
  plt.xlabel("X distance to target")
  plt.ylabel("Y distance to target")
  plt.show()


plot_simulations(xy=th.detach(xy), target_xy=th.detach(tg))